In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('/content/IITH-DMU/labels.csv')
df.head()

,filename,width,height,xmin,ymin,xmax,ymax,class
0,hp_1.jpg,3000,4000,958,1838,1649,2395,Comp
1,hp_10.jpg,4000,3000,2269,774,2582,1084,Comp
2,hp_100.jpg,4624,2136,1110,652,1497,1003,Comp
3,hp_101.jpg,4624,2136,942,723,1365,1200,Comp
4,hp_101.jpg,4624,2136,1910,823,2378,1258,Comp


In [ ]:
df.drop(['width','height','class'],axis=1, inplace=True)

In [ ]:
df.head()

,filename,xmin,ymin,xmax,ymax
0,hp_1.jpg,958,1838,1649,2395
1,hp_10.jpg,2269,774,2582,1084
2,hp_100.jpg,1110,652,1497,1003
3,hp_101.jpg,942,723,1365,1200
4,hp_101.jpg,1910,823,2378,1258


In [ ]:
import torchvision
import torch

In [ ]:

from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import random
from collections import Counter

In [ ]:
class CustData(torch.utils.data.Dataset):
  def __init__(self,df,unique_imgs,indices):
    self.df=df
    self.unique_imgs=unique_imgs
    self.indices=indices
  def __len__(self):
    return len(self.indices)
  def __getitem__(self, idx):
    image_name=self.unique_imgs[self.indices[idx]]
    boxes=self.df[self.df.filename == image_name].values[:,1:].astype('float16')
    img=Image.open('./dataset/image/'+ image_name).convert('RGB')
    labels=torch.ones((boxes.shape[0]),dtype=torch.int64)
    target={}
    target['boxes']=torch.tensor(boxes)
    target['label']=labels
    return T.ToTensor()(img),target
    # T.ToTensor()(img.resize((round(img.size[0]*0.25), round(img.size[1]*0.25)))),target

In [ ]:
unique_imgs=df['filename'].unique()

In [ ]:
train_inds,val_inds= train_test_split(range(unique_imgs.shape[0]),test_size=.1)

In [ ]:
def custom_collate(data):
  return data

In [ ]:
from sklearn.utils import shuffle
train_dl=torch.utils.data.DataLoader(CustData(df, unique_imgs,train_inds),
                                      batch_size=2,
                                      shuffle=True,
                                      collate_fn=custom_collate,
                                      pin_memory=True if torch.cuda.is_available() else False)
val_dl=torch.utils.data.DataLoader(CustData(df, unique_imgs,val_inds),
                                      batch_size=1,
                                      shuffle=True,
                                      collate_fn=custom_collate,
                                      pin_memory=True if torch.cuda.is_available() else False)

In [ ]:
model=torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes=2
in_features=model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor= FastRCNNPredictor(in_features,num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cpu')

In [ ]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.001, momentum=0.9,weight_decay=.0005)
num_epochs= 10

In [ ]:
model.to(device)
for epochs in range(num_epochs):
  epoch_loss = 0
  for data in train_dl:
    imgs = []
    targets = []
    for d in data:
      imgs.append(d[0].to(device))
      targ={}
      targ['boxes']=d[1]['boxes'].to(device)
      targ['labels']=d[1]['label'].to(device)
      targets.append(targ)
    loss_dict=model(imgs,targets)
    loss= sum(v for v in loss_dict.values())
    epoch_loss+=loss.cpu().detach().numpy()
    optimizer.zero_grad()
    optimizer.step()
  print(epoch_loss)


282.23427444696426
302.91474467515945
293.94099670648575


In [ ]:
torch.save(model,'/content/drive/MyDrive/MobilePhoneUsage/model4.pt')